In [2]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

In [3]:
headers = {}

# try except решениe gpt для избежания ошибок в парсере
def get_wine_data(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")
    
    name = year = country = sugar = price = rating = review_count = None
    
    try:
        head = soup.find('div', class_='product-page__head')
        head_info = head.find('h1', class_='product-page__header').text.strip().split(', ')
        name = ', '.join(head_info[:-1])
        year = int(head_info[-1].replace(' г.', ''))
    except:
        pass
    
    try:
        country_elem = soup.select_one("dd.product-brief__value")
        if country_elem:
            country = country_elem.get_text(strip=True).split(", ")[0]
    except:
        pass
    
    try:
        sugar_elem = soup.select("dd.product-brief__value")[3]
        if sugar_elem:
            sugar = sugar_elem.get_text(strip=True).split(": ")[1].capitalize()
    except:
        pass
    
    try:
        price = soup.find('div', class_='product-buy__price').text.strip().split()
        price = ''.join(price[:2]).replace('₽', '')
        price = int(price)
    except:
        pass
    
    try:
        rating_box = soup.select_one("div.rating-stars__wrapper.js-product-scroll-to")
        if rating_box:
            rating = float(rating_box.select_one("p.rating-stars__value").get_text(strip=True).replace(",", "."))
            review_count_elem = rating_box.select_one("span.rating-stars__reviews-count")
            if review_count_elem:
                review_count = int(review_count_elem.get_text(strip=True).split()[0])
    except:
        pass
        
    return [name, year, country, sugar, price, rating, review_count]

In [4]:
def get_wine_urls(page):
    urls = []
    url = f"https://simplewine.ru/catalog/vino/page{page}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")
    wine_items = soup.find_all("div", class_="catalog-grid__item")
    for wine_item in wine_items:
        wine_url = "https://simplewine.ru" + wine_item.find("a").get("href")
        urls.append(wine_url)
    return urls

In [5]:
def save_to_csv(data, filename):
    with open(filename, "a+", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Имя", "Год", "Страна", "Сахар", "Цена", "Рейтинг", "Количество отзывов"])
        writer.writerows(data)

In [7]:
if __name__ == "__main__": # gpt придумал для оптимизации
    data = []
    for page in range(1, 41):
        urls = get_wine_urls(page)
        for url in urls:
            wine_data = get_wine_data(url)
            data.append(wine_data)
            time.sleep(2)
    save_to_csv(data, "simplewine.csv")